# Laboratorio 7 - Seguridad en modelos de data science

In [123]:
!pip install scikit-learn==1.1.0

"pip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [124]:
!pip install adversarial-robustness-toolbox

"pip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [167]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras.constraints import maxnorm
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from art.estimators.classification import KerasClassifier
from art.attacks import ExtractionAttack
from art.attacks.extraction import CopycatCNN, KnockoffNets
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, Reshape
from art.attacks.evasion import FastGradientMethod
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from art.defences.postprocessor import ReverseSigmoid
from art.defences.preprocessor import SpatialSmoothing

In [126]:
DATA_DIRECTORY = './B'

###Carga de imagenes

In [127]:
tf.compat.v1.disable_eager_execution()

In [128]:
path_images = ImageDataGenerator().flow_from_directory(directory=DATA_DIRECTORY, target_size=(128,128), batch_size=10000)

Found 9339 images belonging to 25 classes.


In [129]:
images, marks = next(path_images)

In [130]:
X_train, X_test, y_train, y_test = train_test_split(images, marks, test_size = 0.3)

In [131]:
X_train.shape

(6537, 128, 128, 3)

In [132]:
y_train.shape

(6537, 25)

In [133]:
X_train.size

321306624

In [134]:
y_train.size

163425

In [135]:
X_test.shape

(2802, 128, 128, 3)

In [136]:
y_test.shape

(2802, 25)

In [137]:
X_test.size

137723904

In [138]:
y_test.size

70050

In [139]:
print('Images size --> ', images.itemsize)
print('Marks --> ',marks.itemsize)

Images size -->  4
Marks -->  4


In [140]:
images

array([[[[  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],

        [[  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],

        [[  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],

        ...,

        [[ 97.,  97.,  97.],
         [  4.,   4.,   4.],
         [187., 187., 187.],
         ...,
         [ 64.,  64.,  64.],
         [ 74.,  74.,  74.],
         [ 53.,  53.,  53.]],

        [[ 96.,  96.,  96.],
         [200., 200., 200.],
         [165., 165., 165.],
         ...,
         [118., 118., 118.],
         [217., 217., 217.],
         [155., 155., 155.]],

        [[149., 149., 149.],
       

In [141]:
marks

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

###Modelo

In [212]:
model = Sequential()
model.add(keras.layers.Rescaling(1./255, input_shape=(128, 128, 3)))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dense(25, activation='softmax'))

In [219]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [144]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 128, 128, 3)       0         
                                                                 
 conv2d_30 (Conv2D)          (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 63, 63, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_31 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 30, 30, 64)       0         
 g2D)                                                            
                                                                 
 flatten_15 (Flatten)        (None, 57600)           

In [220]:
model

In [221]:
model.layers

In [222]:
model.metrics

In [224]:
op = model.evaluate(X_test, y_test)
print("Accuracy ", op[1])

Accuracy  0.021413276


In [149]:
model.fit(X_train, y_train, batch_size=10, epochs=5)

Train on 6537 samples
Epoch 1/5
6537/6537 [==============================] - 143s 22ms/sample - loss: 0.6157 - accuracy: 0.8856
Epoch 2/5
6537/6537 [==============================] - 141s 22ms/sample - loss: 0.1365 - accuracy: 0.9644
Epoch 3/5
6537/6537 [==============================] - 140s 21ms/sample - loss: 0.0749 - accuracy: 0.9772
Epoch 4/5
6537/6537 [==============================] - 140s 21ms/sample - loss: 0.0661 - accuracy: 0.9810
Epoch 5/5
6537/6537 [==============================] - 138s 21ms/sample - loss: 0.0635 - accuracy: 0.9832


In [150]:
print(model.evaluate(X_test, y_test))

[0.16058630723443726, 0.95431834]


In [151]:
model.save('./B/modelo.h5')

Ataque de evasion

In [152]:
Modelmalware = KerasClassifier(model, clip_values=(0, 1), use_logits=False)

In [153]:
x_test_pred = np.argmax(Modelmalware.predict(X_test), axis=1)
nb_correct_pred = np.sum(x_test_pred == np.argmax(y_test, axis=1))

print("Clasificados correctamente: {}".format(nb_correct_pred))
print("Clasificados incorrectamente: {}".format(len(X_test)-nb_correct_pred))
print("Porcentaje: {}".format(nb_correct_pred/len(X_test)))

c:\Users\owdel\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Clasificados correctamente: 2674
Clasificados incorrectamente: 128
Porcentaje: 0.9543183440399714


In [154]:
attacker = FastGradientMethod(Modelmalware, eps=1.7)
x_test_adv = attacker.generate(X_test)

In [155]:
y_pred_adv = Modelmalware.predict(x_test_adv)
accuracy_adv = np.mean(np.argmax(y_test, axis=1) == np.argmax(y_pred_adv, axis=1))
print("Accuracy de los ejemplos adversarios:", accuracy_adv)
perturbation = np.mean(np.abs(x_test_adv - X_test))
print("Perturbación media:", perturbation)

Accuracy de los ejemplos adversarios: 0.9564596716630978
Perturbación media: 1.5572315


Defensa de evasion

In [169]:
defender = SpatialSmoothing(window_size=3)
x_def, _ = defender(images)
x_adv_def, _ = defender(x_test_adv)



In [171]:
y_predict = model.predict(X_test)
y_adv_predict = model.predict(x_adv_def)

In [191]:
accuracy = np.mean(y_test == y_predict)
accuracy2 = np.mean(y_test == y_adv_predict)
print("Precisión de original:", accuracy)
print("Precision de adversario: ", accuracy2)

Precisión de original: 0.0073661670235546034
Precision de adversario:  0.0012705210563882941


Ataque de extraccion

In [156]:
Modelmalware1 = KerasClassifier(model)

In [157]:
len_steal = 2000
epochs = 5

In [158]:
stealed = Sequential()
stealed.add(keras.layers.Rescaling(1./255, input_shape=(128, 128, 3)))
stealed.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
stealed.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
stealed.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
stealed.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
stealed.add(keras.layers.Flatten())
stealed.add(keras.layers.Dense(1024, activation='relu'))
stealed.add(keras.layers.Dense(25, activation='softmax'))

In [159]:
stealed.compile(optimizer="adam",loss="categorical_crossentropy", metrics=["accuracy"])

In [160]:
stealed = KerasClassifier(stealed, clip_values=(0, 1), use_logits=False)

In [161]:
copycat = CopycatCNN(nb_epochs=epochs, nb_stolen=len_steal, use_probability=True, classifier=Modelmalware1)


In [162]:
stealedC = copycat.extract(x=X_train, y=y_train, thieved_classifier=stealed)

Train on 2000 samples
Epoch 1/5
2000/2000 [==============================] - 269s 134ms/sample - loss: 0.8991 - accuracy: 0.8055
Epoch 2/5
2000/2000 [==============================] - 330s 165ms/sample - loss: 0.2713 - accuracy: 0.9425
Epoch 3/5
2000/2000 [==============================] - 287s 144ms/sample - loss: 0.2095 - accuracy: 0.9635
Epoch 4/5
2000/2000 [==============================] - 664s 332ms/sample - loss: 0.1614 - accuracy: 0.9785
Epoch 5/5
2000/2000 [==============================] - 401s 201ms/sample - loss: 0.1428 - accuracy: 0.9850


In [165]:
y_predic = stealedC.predict(X_test)


In [166]:
accuracy_ = np.mean(np.argmax(y_test, axis=1) == np.argmax(y_predic, axis=1))
print("Accuracy de los ejemplos adversarios:", accuracy_)


Accuracy de los ejemplos adversarios: 0.9389721627408993


Defensa de extraccion

In [199]:
post = ReverseSigmoid(
  beta=1.0,
  gamma=0.2
)

defmod = KerasClassifier(
  model=model,
  postprocessing_defences=post,
)

In [200]:
stlmodel = Sequential()
stlmodel.add(keras.layers.Rescaling(1./255, input_shape=(128, 128, 3)))
stlmodel.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
stlmodel.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
stlmodel.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
stlmodel.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
stlmodel.add(keras.layers.Flatten())
stlmodel.add(keras.layers.Dense(1024, activation='relu'))
stlmodel.add(keras.layers.Dense(25, activation='softmax'))

In [201]:
stlmodel.compile(optimizer="adam",loss="categorical_crossentropy", metrics=["accuracy"])

In [203]:
StoleModel = KerasClassifier(stlmodel)

copycat_cnn_def = CopycatCNN(
  batch_size_fit=16,
  batch_size_query=16,
  nb_epochs=5,
  nb_stolen=len(X_train),
  classifier=defmod
)

stolenModel = copycat_cnn_def.extract(
  x=X_train, 
  y=y_train, 
  thieved_classifier=StoleModel
)

c:\Users\owdel\AppData\Local\Programs\Python\Python311\Lib\site-packages\art\defences\postprocessor\reverse_sigmoid.py:75: RuntimeWarning: divide by zero encountered in log
  perturbation_r = self.beta * (sigmoid(-self.gamma * np.log((1.0 - preds_clipped) / preds_clipped)) - 0.5)


Train on 6537 samples
Epoch 1/5
6537/6537 [==============================] - 80s 12ms/sample - loss: 0.0109 - accuracy: 0.9965
Epoch 2/5
6537/6537 [==============================] - 79s 12ms/sample - loss: 0.0193 - accuracy: 0.9931
Epoch 3/5
6537/6537 [==============================] - 82s 13ms/sample - loss: 0.0016 - accuracy: 0.9997
Epoch 4/5
6537/6537 [==============================] - 90s 14ms/sample - loss: 0.0627 - accuracy: 0.9829
Epoch 5/5
6537/6537 [==============================] - 84s 13ms/sample - loss: 0.0445 - accuracy: 0.9907


In [209]:
y_predict = model.predict(X_test)
y_adv_predict = stolenModel.predict(x_adv_def)

In [210]:
accuracy = np.mean(y_test == y_predict)
accuracy2 = np.mean(y_test == y_adv_predict)
print("Precisión de original:", accuracy)
print("Precision de adversario: ", accuracy2)

Precisión de original: 0.0073661670235546034
Precision de adversario:  0.0013990007137758744


Disución de resultados:

Como podemos ver en las metricas obtenidas, en la creacion del modelo se obtuvo un acurancy de 0.0214, luego del fit al modelo el acurancy subio a 0.9543.

En el caso del ataque de evasion se obtuvo un accurancy de 0.9564, y en el caso del ataque de extrancion un accurancy de 0.9389.
Ademas del acurancy se hace una comparacion ante los ejemplos adversarios, en el caso de los dos ataques este numero supera el 0.9, en uno llega a 0.95 lo que nos dice que los datos obtentenidos en los ataques llegan a ser casi identicos a los originales. 

En los casos de implementación de defensa, pudimos observar una baja considerable en los modelos adversarios, en caso de ataque de evasion, la precision llego solamente a 0.0012, en el caso del ataque de extraccion la precisión llego solamnete a 0.0013 teniendo antes una precisión de 0.95 y 0.93 respectivamente. Por lo tanto, se puede afirmar que los metodos de defensa implementados fueron efectivos.
